In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import pandas as pd
import scipy.stats
import math

In [290]:
def getMergedDataFrameFast(typeof):
    truth = pd.read_table(files[typeof]["truth"], delim_whitespace=True, \
       usecols=["transcript_id", "count"])
    df = truth
    df.rename(index=str, \
            columns={"transcript_id": "Name", "count": "count"}, inplace = True)
    
    sla = pd.read_table(files[typeof]["SLA"], delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])
    df = pd.merge(df,sla, how="outer", on = "Name").fillna(0.0)
    
    kallisto = pd.read_table(files[typeof]["kallisto"], delim_whitespace=True, \
                                     usecols=["target_id", "est_counts"])
    kallisto.rename(index=str, columns={"target_id": "Name", \
                                                "est_counts": "NumReads_KAL"}, inplace = True)
    df = pd.merge(df,kallisto, how="outer", on = "Name").fillna(0.0)
    
    hera = pd.read_table(files[typeof]["hera"], delim_whitespace=True, \
                                     usecols=["#target_id", "est_counts"])
    hera["#target_id"]= hera["#target_id"].str.split(":",expand=True)[0]
    hera.rename(index=str, columns={"#target_id": "Name", \
                                                "est_counts": "NumReads_hera"}, inplace = True)
    df = pd.merge(df,hera, how="outer", on = "Name").fillna(0.0)
    return (df,truth,sla,kallisto,hera)

In [284]:
def getMergedDataFrame(typeof):
    df = pd.read_table(files[typeof]["truth"], delim_whitespace=True, \
                                  usecols=["transcript_id", "count"])
    df.rename(index=str, \
            columns={"transcript_id": "Name", "count": "count"}, inplace = True)
    
    sla = pd.read_table(files[typeof]["SLA"], delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])
    df = pd.merge(df,sla, how="outer", on = "Name").fillna(0.0)
    
    kallisto = pd.read_table(files[typeof]["kallisto"], delim_whitespace=True, \
                                     usecols=["target_id", "est_counts"])
    kallisto.rename(index=str, columns={"target_id": "Name", \
                                                "est_counts": "NumReads_KAL"}, inplace = True)
    df = pd.merge(df,kallisto, how="outer", on = "Name").fillna(0.0)
    
    hera = pd.read_table(files[typeof]["hera"], delim_whitespace=True, \
                                     usecols=["#target_id", "est_counts"])
    hera["#target_id"]= hera["#target_id"].str.split(":",expand=True)[0]
    hera.rename(index=str, columns={"#target_id": "Name", \
                                                "est_counts": "NumReads_hera"}, inplace = True)
    df = pd.merge(df,hera, how="outer", on = "Name").fillna(0.0)
    
    star = pd.read_table(files[typeof]["star"], delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])
    star.rename(index = str, columns={"Name": "Name", \
                                              "NumReads": "NumReads_STAR"}, inplace = True)
    
    
    df = pd.merge(df,star, how="outer", on = "Name").fillna(0.0)
                         
    bt = pd.read_table(files[typeof]["bowtie"], delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])
    bt.rename(index = str, columns={"Name": "Name", \
                                              "NumReads": "NumReads_BT"}, inplace = True)
    df = pd.merge(df,bt, how="outer", on = "Name").fillna(0.0)
    return df

In [345]:
prefix = "/mnt/scratch6/hirak/SLA_benchmark/bleed_through/"
types = ["30_percent"]
files = {}
for t in types:
    fileinfo = {}
    fileinfo["truth"]= "/".join([prefix,t,"rsem_sim","sim.sim.isoforms.results"])
    fileinfo["SLA"] =  "/".join([prefix,t,"quant","salmon_sl","quant.sf"])
    fileinfo["kallisto"] = "/".join([prefix,t,"quant","kallisto_out","abundance.tsv"])
    fileinfo["hera"] = "/".join([prefix,t,"quant","hera_out","abundance.tsv"])
    fileinfo["star"] = "/".join([prefix,t,"quant","star_out","quant.sf"])
    fileinfo["bowtie"] = "/".join([prefix,t,"quant","bowtie_out","quant.sf"])
    files[t] = fileinfo

In [346]:
fileinfo

{'SLA': '/mnt/scratch6/hirak/SLA_benchmark/bleed_through//30_percent/quant/salmon_sl/quant.sf',
 'bowtie': '/mnt/scratch6/hirak/SLA_benchmark/bleed_through//30_percent/quant/bowtie_out/quant.sf',
 'hera': '/mnt/scratch6/hirak/SLA_benchmark/bleed_through//30_percent/quant/hera_out/abundance.tsv',
 'kallisto': '/mnt/scratch6/hirak/SLA_benchmark/bleed_through//30_percent/quant/kallisto_out/abundance.tsv',
 'star': '/mnt/scratch6/hirak/SLA_benchmark/bleed_through//30_percent/quant/star_out/quant.sf',
 'truth': '/mnt/scratch6/hirak/SLA_benchmark/bleed_through//30_percent/rsem_sim/sim.sim.isoforms.results'}

In [347]:
df_30 = getMergedDataFrame("30_percent")

In [348]:
print("kallisto",df_30["NumReads_BT"].corr(df_30["NumReads_KAL"],method="spearman"))
print("SLA",df_30["NumReads_BT"].corr(df_30["NumReads"],method="spearman"))
print("hera",df_30["NumReads_BT"].corr(df_30["NumReads_hera"],method="spearman"))
print("star",df_30["NumReads_BT"].corr(df_30["NumReads_STAR"],method="spearman"))
print("bt",df_30["NumReads_BT"].corr(df_30["NumReads_BT"],method="spearman"))

('kallisto', 0.85011685118902125)
('SLA', 0.81599933444666339)
('hera', 0.89034634561535275)
('star', 0.9356914460652378)
('bt', 1.0)


In [329]:
df_30['ARD_ratio'] = ((np.abs(df_30["NumReads_BT"] - df_30["NumReads"])+1) \
 /(np.abs(df_30["NumReads_BT"] - df_30["NumReads_hera"])+1)).fillna(0)

In [341]:
df_30.sort_values(['ARD_ratio'],ascending=False)

,Name,count,NumReads,NumReads_KAL,NumReads_hera,NumReads_STAR,NumReads_BT,ARD_ratio
23100,ENST00000620821.1,0.0,1624.614701,1262.130000,0.000000,104.317498,0.000000,1625.614701
181633,ENST00000597934.1,0.0,943.048231,185.400000,4.467358,0.000000,0.000000,172.669913
172361,ENST00000580177.5,0.0,201.227967,0.000483,0.281709,0.377491,0.101174,171.216265
73582,ENST00000523921.1,0.0,129.618970,0.000000,0.000000,0.000000,0.000000,130.618970
23502,ENST00000426482.3,0.0,128.525596,40.348200,0.000000,5.005367,0.000000,129.525596
126987,ENST00000421138.6,0.0,120.513992,15.726400,0.000000,0.000000,0.000000,121.513992
190607,ENST00000599247.5,0.0,97.448457,4.195100,0.000000,0.000000,0.000000,98.448457
23411,ENST00000619423.4,0.0,90.321071,0.000000,0.000000,0.000000,0.000000,91.321071
89768,ENST00000402030.6,0.0,10213.527172,10230.200000,10307.760962,9850.660032,10307.929538,81.639847
5858,ENST00000258711.7,3201.0,3084.653181,2773.410000,3180.966821,3176.051993,3181.341625,71.056270


In [161]:
print("kallisto",len(df_30[(df_30['count'] == 0) & (df_30['NumReads_KAL'])]))
print("SLA",len(df_30[(df_30['count'] == 0) & (df_30['NumReads'])]))
print("hera",len(df_30[(df_30['count'] == 0) & (df_30['NumReads_hera'])]))
print("star",len(df_30[(df_30['count'] == 0) & (df_30['NumReads_STAR'])]))
print("bt",len(df_30[(df_30['count'] == 0) & (df_30['NumReads_BT'])]))

('kallisto', 6987)
('SLA', 4018)
('hera', 3900)
('star', 3962)
('bt', 3597)


In [281]:
prefix = "/mnt/scratch6/hirak/SLA_benchmark/bleed_through/"
types = ["30_percent"]
files = {}
for t in types:
    fileinfo = {}
    fileinfo["truth"]= "/".join([prefix,t,"rsem_sim","sim.sim.isoforms.results"])
    fileinfo["SLA"] =  "/".join([prefix,t,"quant","salmon_out","quant.sf"])
    fileinfo["kallisto"] = "/".join([prefix,t,"quant","kallisto_out","abundance.tsv"])
    fileinfo["hera"] = "/".join([prefix,t,"quant","hera_out","abundance.tsv"])
    files[t] = fileinfo

In [291]:
df_30, truth, sla,kallisto,hera = getMergedDataFrameFast("30_percent")

In [305]:
set(truth.Name.values).difference(set(sla.Name.values))

{'ENST00000517568.4',
 'ENST00000462877.4',
 'ENST00000457805.5',
 'ENST00000457316.4',
 'ENST00000530454.4',
 'ENST00000556783.4',
 'ENST00000592202.4',
 'ENST00000429662.5',
 'ENST00000322954.9',
 'ENST00000514972.4',
 'ENST00000337665.7',
 'ENST00000383738.5',
 'ENST00000513206.4',
 'ENST00000402239.6',
 'ENST00000418055.4',
 'ENST00000614486.3',
 'ENST00000536290.4',
 'ENST00000351660.8',
 'ENST00000572182.4',
 'ENST00000462224.4',
 'ENST00000505993.4',
 'ENST00000559048.4',
 'ENST00000023064.7',
 'ENST00000466993.4',
 'ENST00000470476.4',
 'ENST00000262764.9',
 'ENST00000389629.7',
 'ENST00000254262.8',
 'ENST00000371217.8',
 'ENST00000521943.4',
 'ENST00000421999.5',
 'ENST00000291416.8',
 'ENST00000417398.4',
 'ENST00000450491.4',
 'ENST00000405816.4',
 'ENST00000511336.4',
 'ENST00000340490.6',
 'ENST00000485326.5',
 'ENST00000392826.5',
 'ENST00000555647.4',
 'ENST00000374555.6',
 'ENST00000535355.4',
 'ENST00000407170.5',
 'ENST00000412265.4',
 'ENST00000468577.4',
 'ENST0000

In [288]:
print("kallisto",df_30["count"].corr(df_30["NumReads_KAL"],method="spearman"))
print("SLA",df_30["count"].corr(df_30["NumReads"],method="spearman"))
print("hera",df_30["count"].corr(df_30["NumReads_hera"],method="spearman"))

('kallisto', 0.1436230151626629)
('SLA', 0.16897107007145784)
('hera', 0.16744798364338395)


In [250]:
salmon_quant_file = "/mnt/scratch4/SLA-benchmarking/clean-benchmarks/selective-alignment-experiment/\
quant_scripts/samples/ERR188140/result.salmon.k31/quant.sf" 

In [251]:
quant = pd.read_table(salmon_quant_file, delim_whitespace=True, \
                                 usecols=["Name", "NumReads"])

In [252]:
quant["Name_clean"] = quant["Name"].str.split(":",expand=True)[0]

In [253]:
tid30 = pd.read_table("/mnt/scratch6/hirak/SLA_benchmark/refpc/tid_60.txt",header=None,names=['tid'])

In [254]:
quant.loc[~quant['Name_clean'].isin(tid30.tid.values),'NumReads'] = 0.0

In [255]:
quant.columns = ['transcript_id','count','to_drop']

In [256]:
quant['count'] = np.ceil(quant['count'])

In [257]:
len(quant[quant['count'] != 0])

33388

In [258]:
len(quant)

199851

In [259]:
to_keep = list(quant['transcript_id'].values)

In [260]:
len(to_keep)

199851

In [261]:
new_tr = pd.read_table("/mnt/scratch4/SLA-benchmarking/clean-benchmarks/selective-alignment-experiment/quant_scripts/salmon.index.31/duplicate_clusters.tsv")

In [262]:
df_keep = pd.DataFrame()
df_keep['transcript_id'] = new_tr['DuplicateTxp'] 

In [263]:
df_keep['count'] = 0.0

In [264]:
df_keep['to_drop'] = None

In [265]:
quant2 = quant.append(df_keep)

In [266]:
quant2.to_csv("/mnt/scratch6/hirak/SLA_benchmark/polyester_bleedthrough/poly.model.60.sf",\
             columns=['transcript_id','count'],sep="\t", index=False)

In [267]:
len(quant2)

200401